Note:
Để chạy Email Daily Sales:
1. Update data OKR daily lúc 4h20 
File này để update data OKR / sheet "Data SKU" => Công thức sẽ tự nhảy cho các sheet Tar OKR Jan 24 > OKR Order Category Jan 24 > Target OKR Jan 24
https://docs.google.com/spreadsheets/d/1WtDK3oV0wJGgloKW_xCZ2PhyLGRQ72GJYzwEDWaMQB4/edit#gid=786949489


2. Từ sheet Target OKR Jan 24, kéo data lên email để transform và alert (dùng file Mail Daily Sales.ipynb)
- Note: Hàng tháng, cần update tên sheet. Luôn giữ lại 1 tháng để back up

3. Trên email alert: 
- AVC: Data source SSO - chị Linh (những ngày full day data) + RTS (BI INNONET crawl for AMZ only)
- ASC: Data source SSO - chị Linh
- WF: Data source SSO -  chị Linh (chấp nhận trễ vài tiếng vì
    + Data hourly crawled by DnD không có gmv_usd
    + Trễ vài tiếng là do data team SSO sync 2 lần/ngày)
- WM: Data source full từ hourly crawled by DnD (vì có đủ data) (SSO bị delay 1 ngày)
Note:
1. Hiện tại ngoại trừ Services Hoàng Thông, các Services còn lại chưa có OKR => Data SKU hiện tại chưa lấy cho Services. Sẽ add số sales của Services trong file Mail Daily Sales.ipynb để add lên email 
2. Isuse 1: Label time sai => Data sai. Solutions: Check max date <= today - 3
3. Issue 2: SIC input thiếu ASIN vào launching plan => Bổ sung

In [2]:
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
import gspread
import mysql.connector
import pandas as pd
import time
import schedule
import os
import sys
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import datetime
# Local Time
localtime = time.asctime(time.localtime(time.time()))
# Connect Portgresat
import psycopg2

# Kết nối GG Sheet
# Link Credentials: C:/Users/hieptn/Desktop/Test Py/my-project-123-378311-71b37cb67041.json
creds = Credentials.from_service_account_file(r"C:\Users\antv\Desktop\Test Python\test-bot-415102-1878dc99ca19.json")
# Link Google Sheeet: https://docs.google.com/spreadsheets/d/1eszgp5vDiC01Hj89C9xGnesXY3O_k8lzMHxM_jX5q6U/edit#gid=0
service = build('sheets', 'v4', credentials=creds)

# sheet_analys = '1P8nbufU4BtvXeBc-pkzyEXXCtTDsAT8zBKYDBuz-usE'
#https://docs.google.com/spreadsheets/d/1P8nbufU4BtvXeBc-pkzyEXXCtTDsAT8zBKYDBuz-usE/edit#gid=1109562276
sheet_OKR = '173CwhjxOkark9N4UTJJiiD6a7eN6IvUWcA4zDLs5l1w'
#https://docs.google.com/spreadsheets/d/173CwhjxOkark9N4UTJJiiD6a7eN6IvUWcA4zDLs5l1w/edit#gid=2125877975

def MY_SQL_TO_GG(df_mysql,sheet_id,range_name,delete_range=None): # đẩy data từ sql sang gg sheet
    localtime = time.asctime(time.localtime(time.time()))
    date = [["Date update", localtime]]
    # Covert to list
    df_mysql = df_mysql.astype(str)
    header = [list(df_mysql.columns.values)]
    values = df_mysql.values.tolist()
    # SQL 
    # Delete old data
    request = service.spreadsheets().values().clear(
    spreadsheetId = sheet_id,
    range = range_name + delete_range, 
    body={}).execute()
    # Write local date
    result_1 = service.spreadsheets().values().update(
    spreadsheetId = sheet_id,      
    range = range_name + '!A1',
    valueInputOption='USER_ENTERED', body={'values': date}).execute()
    # Write header column
    result_2 = service.spreadsheets().values().update(
    spreadsheetId = sheet_id,      
    range = range_name + '!A2',
    valueInputOption='USER_ENTERED', body={'values': header}).execute()
    # Write data
    result_3 = service.spreadsheets().values().update(
    spreadsheetId = sheet_id,      
    range = range_name + '!A3',
    valueInputOption='USER_ENTERED', body={'values': values}).execute()  

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [18]:
def Analyst_sales(): # lấy data từ DB + call def đẩy data lên gg sheet
    now = datetime.datetime.now()
    day = datetime.timedelta(days=1)
    data_date = now - day
    data_time = data_date.strftime("%Y%m%d")

    #### Lấy data postgresql
    conn_post = psycopg2.connect(host="172.30.105.112",database="y4a_datamart",user="y4a_dl_vylm",password="ACB!asdlkj123")
    # Data BI (Quang Ù)
    conn_post_1 = psycopg2.connect(host="172.29.15.98",database="mi_report", user="da_report",password="Inno202$")
    # Lấy list Asin, SKU
    list_asin_sku = pd.read_sql_query(f"""
        select asin, sku from public.launching_plan where asin is not null and sku is not null
        """,conn_post_1) 
     # List_asin
    list_asin = tuple(list_asin_sku['asin'])
    # List_sku
    list_sku = tuple(list_asin_sku['sku'])
    df_max_date_real_sales = pd.read_sql(f'''
        select MAX(log_date) as MAX_DAY from y4a_sop_sell_out.sale_performance_by_channel_dpl
        where channel in ('AVC WH','AVC DI') and label_datetime = 'daily' and asin in {list_asin}  
    ''', con=conn_post)
    # and log_date <= '2023-12-07'
    MY_SQL_TO_GG(df_max_date_real_sales,sheet_OKR,'Max day Update','!A1:B3')
    print(df_max_date_real_sales)
    df_max_date = df_max_date_real_sales.iloc[0]['max_day']
    max_data_time = df_max_date.strftime("%Y-%m-%d")
    day = datetime.timedelta(days=1)
    
    # Lấy ngày tiếp theo của real sales để lấy real time sales (code phía sau)
    max_data_time_1 = df_max_date + day
    max_data_time_format = max_data_time_1.strftime("%d-%b-%y")
    
    # Lấy full day sales data theo invoice
    # từ bảng public_main.sale_performance_by_channel_innonet 
    # 1. AMZ: độ trễ lớn hơn => max date of data bé hơn
    # 2. WM + WF + ASC: độ trễ nhỏ hơn => max date of data nhỏ hơn (cách hiện tại ~ 1 ngày)
    # do đó phải query riêng và union

    # chỉ query 2 tháng gần nhất
    df_mysql_6_OKR = pd.read_sql(f''' 
        --------------- AVC
        select 
          report_date log_date
          , asin
          , sku
          , CASE WHEN channel like 'WM%' THEN 'Walmart' WHEN channel like 'WF%' THEN 'Wayfair' ELSE 'Amazon' END as channel
          , sum(ordered_qty) as ordered_units, sum(ordered_gmv_usd) as ordered_gmv
          , sum(shipped_qty) as ship_units, sum(shipped_gmv_usd) as ship_gmv
          , 'MySQL' as type
        from y4a_inno.view_fact_dpl_gmv_rev_sell_out_by_sku
        where 1=1
          and sku in {list_sku}
          and report_date <= '{max_data_time}'
          and channel like 'AVC%'
          and country = 'USA'
          and CASE 
            WHEN EXTRACT(month FROM DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')) = 1
            THEN
            (EXTRACT(month FROM DATE_TRUNC('month', report_date)) = 12 AND 
              EXTRACT(year FROM DATE_TRUNC('year', report_date)) = EXTRACT(year FROM DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')) - 1)
              OR 
              (EXTRACT(month FROM DATE_TRUNC('month', report_date)) = 1 AND 
              DATE_TRUNC('year', report_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day'))
            ELSE
              (EXTRACT(month FROM DATE_TRUNC('month', report_date)) = EXTRACT(month FROM DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day'))
              OR (EXTRACT(month FROM DATE_TRUNC('month', report_date)) = EXTRACT(month FROM DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')) - 1)
              OR (EXTRACT(month FROM DATE_TRUNC('month', report_date)) = EXTRACT(month FROM DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')) - 2))
            AND EXTRACT(year FROM DATE_TRUNC('year', report_date)) = EXTRACT(year FROM DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day'))
            END
        group by 
          report_date
          , asin
          , sku
          , channel

        
        Union all

      --------------------- WM + WF + ASC
      select 
        report_date
        , asin
        , sku
        , CASE WHEN channel like 'WM%' THEN 'Walmart' WHEN channel like 'WF%' THEN 'Wayfair' ELSE 'Amazon' END as channel
        , sum(ordered_qty) as ordered_units,sum(ordered_gmv_usd) as ordered_gmv
        , sum(shipped_qty) as ship_units, sum(shipped_gmv_usd) as ship_gmv
        , 'MySQL' as type
      from y4a_inno.view_fact_dpl_gmv_rev_sell_out_by_sku
      where 1=1
        and sku in {list_sku}
        and report_date <= '{data_time}'
        and channel not like 'AVC%'
        and country = 'USA'
        and CASE 
          WHEN EXTRACT(month FROM DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')) = 1
          THEN
          (EXTRACT(month FROM DATE_TRUNC('month', report_date)) = 12 AND 
            EXTRACT(year FROM DATE_TRUNC('year', report_date)) = EXTRACT(year FROM DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')) - 1)
            OR 
            (EXTRACT(month FROM DATE_TRUNC('month', report_date)) = 1 AND 
            DATE_TRUNC('year', report_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day'))
          ELSE
            (EXTRACT(month FROM DATE_TRUNC('month', report_date)) = EXTRACT(month FROM DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day'))
            OR (EXTRACT(month FROM DATE_TRUNC('month', report_date)) = EXTRACT(month FROM DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')) - 1)
            OR (EXTRACT(month FROM DATE_TRUNC('month', report_date)) = EXTRACT(month FROM DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')) - 2))
          AND EXTRACT(year FROM DATE_TRUNC('year', report_date)) = EXTRACT(year FROM DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day'))
          END
      group by 
        report_date
        , asin
        , sku
        , channel       
    ''', con=conn_post)
    
    # lấy real time sales từ db INNONET BI
    df_oracle_7_OKR = pd.read_sql_query(f"""
        WITH t1 AS 
          (
          SELECT 
            DATE_TRUNC('day', hour_pt) AS log_date
            , asin
            , SUM(ordered_units) AS ordered_units
            , SUM(ordered_revenue) AS ordered_gmv
            , 0 AS ship_units
            , 0 AS ship_gmv 
          FROM dpl.ara_realtime_sales
          WHERE
              DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day') = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
              AND hour_pt >= '{max_data_time_format}%'
              AND hour_pt < DATE_TRUNC('day', CURRENT_DATE)
          GROUP BY 
            DATE_TRUNC('day', hour_pt) 
            , asin
          ORDER BY 
            log_date DESC
          ), 

          t2 AS 
          (
            SELECT 
              asin
              , SUBSTRING(sku FROM 1 FOR 4) AS SKU 
            FROM public.launching_plan -- SKU mặc định 4 chữ, một số bundle/slob sẽ add thêm chữ/số phía sau
          )
        SELECT
            log_date
            , t1.asin
            , t2.sku
            , SUM(ordered_units) AS ordered_units
            , SUM(ordered_gmv) AS ordered_gmv
            , 0 AS ship_units
            , 0 AS ship_gmv
            , 'PostgreSQL' AS type
            , 'Amazon' AS channel
        FROM t1
        LEFT JOIN t2 ON t1.asin = t2.asin
        GROUP BY 
          log_date
          , t1.asin
          , t2.sku
        ORDER BY 
          log_date DESC;
    """,conn_post_1)
    df_oracle_7_OKR.columns = df_oracle_7_OKR.columns.str.lower()
    df_OKR = pd.concat([df_mysql_6_OKR, df_oracle_7_OKR])
    print(df_OKR.columns)
    MY_SQL_TO_GG(df_OKR,sheet_OKR,'Data SKU','!A:I')
    print("Done df_OKR")   
    print("Job executed at", datetime.datetime.now())
    print("Đã Update Data Daily Thành Công")

In [19]:
Analyst_sales()

      max_day
0  2024-02-23
Index(['log_date', 'asin', 'sku', 'channel', 'ordered_units', 'ordered_gmv',
       'ship_units', 'ship_gmv', 'type'],
      dtype='object')


In [3]:
schedule.every().day.at("16:45").do(Analyst_sales)
while True:
    schedule.run_pending()
    time.sleep(10)

In [18]:
def Analyst_sales1(): # lấy data từ DB + call def đẩy data lên gg sheet
    now = datetime.datetime.now()
    day = datetime.timedelta(days=1)
    data_date = now - day
    data_time = data_date.strftime("%Y%m%d")

    #### Lấy data postgresql
    conn_post = psycopg2.connect(host="172.30.105.112",database="y4a_datamart",user="y4a_dl_vylm",password="ACB!asdlkj123")
    # Data BI (Quang Ù)
    conn_post_1 = psycopg2.connect(host="172.29.15.98",database="mi_report", user="da_report",password="Inno202$")
    # Lấy list Asin, SKU
    list_asin_sku = pd.read_sql_query(f"""
        select asin, sku from public.launching_plan where asin is not null and sku is not null
        """,conn_post_1) 
     # List_asin
    list_asin = tuple(list_asin_sku['asin'])
    # List_sku
    list_sku = tuple(list_asin_sku['sku'])
    df_max_date_real_sales = pd.read_sql(f'''
        select MAX(log_date) as MAX_DAY from y4a_sop_sell_out.sale_performance_by_channel_dpl
        where channel in ('AVC WH','AVC DI') and label_datetime = 'daily' and asin in {list_asin}  
    ''', con=conn_post) 
    # and log_date <= '2023-12-07'
    MY_SQL_TO_GG(df_max_date_real_sales,sheet_OKR,'Max day Update','!A1:B3')
    print(df_max_date_real_sales)
    df_max_date = df_max_date_real_sales.iloc[0]['max_day']
    max_data_time = df_max_date.strftime("%Y-%m-%d")
    day = datetime.timedelta(days=1)
    
    # Lấy ngày tiếp theo của real sales để lấy real time sales (code phía sau)
    max_data_time_1 = df_max_date + day
    max_data_time_format = max_data_time_1.strftime("%d-%b-%y")
    
    # Lấy full day sales data theo invoice
    # từ bảng public_main.sale_performance_by_channel_innonet 
    # 1. AMZ: độ trễ lớn hơn => max date of data bé hơn
    # 2. WM + WF + ASC: độ trễ nhỏ hơn => max date of data nhỏ hơn (cách hiện tại ~ 1 ngày)
    # do đó phải query riêng và union

    # chỉ query 2 tháng gần nhất
    # df_mysql_6_OKR = pd.read_sql(f''' 
    #     --------------- AVC
    #     select log_date, asin, sku, 
    #     CASE WHEN channel like 'WM%' THEN 'Walmart' WHEN channel like 'WF%' THEN 'Wayfair' ELSE 'Amazon' END as channel,
    #     sum(ordered_units) as ordered_units, sum(ordered_gmv) as ordered_gmv,
    #     sum(shipped_units) as ship_units, sum(shipped_gmv) as ship_gmv,
    #     'MySQL' as type from y4a_sop_sell_out.sale_performance_by_channel_dpl
    #     where sku in {list_sku}
    #     and CASE 
    #     WHEN EXTRACT(month FROM DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')) = 1
    #     THEN
    #      (EXTRACT(month FROM DATE_TRUNC('month', log_date)) = 12 AND 
    #       EXTRACT(year FROM DATE_TRUNC('year', log_date)) = EXTRACT(year FROM DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')) - 1)
    #       OR 
    #       (EXTRACT(month FROM DATE_TRUNC('month', log_date)) = 1 AND 
    #       DATE_TRUNC('year', log_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day'))
    #     ELSE
    #       (EXTRACT(month FROM DATE_TRUNC('month', log_date)) = EXTRACT(month FROM DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day'))
    #       OR (EXTRACT(month FROM DATE_TRUNC('month', log_date)) = EXTRACT(month FROM DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')) - 1)
    #       OR (EXTRACT(month FROM DATE_TRUNC('month', log_date)) = EXTRACT(month FROM DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')) - 2))
    #     AND EXTRACT(year FROM DATE_TRUNC('year', log_date)) = EXTRACT(year FROM DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day'))
    #     END
    #     and log_date <= '{max_data_time}'
    #     and channel like 'AVC%'
    #     and country = 'USA'
    #     group by log_date, asin, sku, channel

        
    #     Union all

    #   --------------------- WM + WF + ASC
    #   select log_date, asin, sku, 
    #     CASE WHEN channel like 'WM%' THEN 'Walmart' WHEN channel like 'WF%' THEN 'Wayfair' ELSE 'Amazon' END as channel,
    #     sum(ordered_units) as ordered_units,sum(ordered_gmv) as ordered_gmv,
    #     sum(shipped_units) as ship_units, sum(shipped_gmv) as ship_gmv,
    #     'MySQL' as type from y4a_sop_sell_out.sale_performance_by_channel_dpl
    #     where sku in {list_sku}
    #     and CASE 
    #     WHEN EXTRACT(month FROM DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')) = 1
    #     THEN
    #      (EXTRACT(month FROM DATE_TRUNC('month', log_date)) = 12 AND 
    #       EXTRACT(year FROM DATE_TRUNC('year', log_date)) = EXTRACT(year FROM DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')) - 1)
    #       OR 
    #       (EXTRACT(month FROM DATE_TRUNC('month', log_date)) = 1 AND 
    #       DATE_TRUNC('year', log_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day'))
    #     ELSE
    #       (EXTRACT(month FROM DATE_TRUNC('month', log_date)) = EXTRACT(month FROM DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day'))
    #       OR (EXTRACT(month FROM DATE_TRUNC('month', log_date)) = EXTRACT(month FROM DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')) - 1)
    #       OR (EXTRACT(month FROM DATE_TRUNC('month', log_date)) = EXTRACT(month FROM DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')) - 2))
    #     AND EXTRACT(year FROM DATE_TRUNC('year', log_date)) = EXTRACT(year FROM DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day'))
    #     END
    #     and log_date <= '{data_time}'
    #     and channel not like 'AVC%'
    #     and country = 'USA'
    #     and label_datetime ='daily'
    #     group by log_date, asin, sku, channel        
    # ''', con=conn_post)
    
    
   

    # lấy real time sales từ db INNONET BI
    df_oracle_7_OKR = pd.read_sql_query(f"""
        WITH t1 AS (
        SELECT DATE_TRUNC('day', hour_pt) AS log_date, asin, SUM(ordered_units) AS ordered_units, 
        SUM(ordered_revenue) AS ordered_gmv, 0 AS ship_units, 0 AS ship_gmv FROM dpl.ara_realtime_sales
        WHERE
            DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day') = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
            AND hour_pt >= '{max_data_time_format}%'
            AND hour_pt < DATE_TRUNC('day', CURRENT_DATE)
        GROUP BY asin, DATE_TRUNC('day', hour_pt)
        ORDER BY log_date DESC
        ), t2 AS (
            SELECT asin, SUBSTRING(sku FROM 1 FOR 4) AS SKU FROM public.launching_plan -- SKU mặc định 4 chữ, một số bundle/slob sẽ add thêm chữ/số phía sau
        )
        SELECT
            log_date, t1.asin, t2.sku, SUM(ordered_units) AS ordered_units, SUM(ordered_gmv) AS ordered_gmv,
            0 AS ship_units, 0 AS ship_gmv, 'PostgreSQL' AS type, 'Amazon' AS channel
        FROM t1
        LEFT JOIN t2 ON t1.asin = t2.asin
        GROUP BY log_date, t1.asin, t2.sku
        ORDER BY log_date DESC;
    """,conn_post_1)
    # df_oracle_7_OKR.columns = df_oracle_7_OKR.columns.str.lower()
    # df_OKR = pd.concat([df_mysql_6_OKR, df_oracle_7_OKR])
    # MY_SQL_TO_GG(df_OKR,sheet_OKR,'Data SKU','!A:I')
    # print("Done df_OKR")   
    # print("Job executed at", datetime.datetime.now())
    # print("Đã Update Data Daily Thành Công")

    print(df_oracle_7_OKR.columns)

In [19]:
Analyst_sales1()

      max_day
0  2024-02-19
Index(['log_date', 'asin', 'sku', 'ordered_units', 'ordered_gmv', 'ship_units',
       'ship_gmv', 'type', 'channel'],
      dtype='object')
